In [ ]:
from concurrent.futures import ProcessPoolExecutor

import numpy as np
from pokerkit import (Automation, Card, Deck, FixedLimitTexasHoldem,
                      StandardHighHand, calculate_equities,
                      calculate_hand_strength, parse_range)
from tqdm import tqdm

In [5]:
# Create a game template (factory) that we can reuse
game = FixedLimitTexasHoldem(
    automations=(
        Automation.ANTE_POSTING,
        Automation.BET_COLLECTION,
        Automation.BLIND_OR_STRADDLE_POSTING,
        Automation.CARD_BURNING,
        Automation.HOLE_DEALING,
        Automation.BOARD_DEALING,
        Automation.HOLE_CARDS_SHOWING_OR_MUCKING,
        Automation.HAND_KILLING,
        Automation.CHIPS_PUSHING,
        Automation.CHIPS_PULLING,
    ),
    ante_trimming_status=True,  # use blinds instead of antes
    raw_antes=0,
    raw_blinds_or_straddles=(2, 4),
    small_bet=4,
    big_bet=8,
    starting_board_count=1,
)

In [6]:
# Simulate 10 hands with random actions
num_hands = 10
starting_stacks = 1000
player_count = 5
current_stacks = [starting_stacks] * player_count  # Track stacks across hands

for hand_num in tqdm(range(num_hands), desc="Simulating hands"):
    print(f"\n{'='*60}")
    print(f"HAND {hand_num + 1} - Button at position {hand_num % player_count}")
    print(f"{'='*60}")

    # Create a fresh state using the game factory
    state = game(
        raw_starting_stacks=current_stacks,
        player_count=player_count,
    )

    print(f"Starting stacks: {state.stacks}")
    print(f"\n--- HOLE CARDS ---")
    for i, hole_cards in enumerate(state.hole_cards):
        print(f"Player {i}: {hole_cards}")

    print(f"\n--- GAME PLAY ---")
    action_count = 0
    last_board_len = 0  # Track board length to detect changes

    # Play the hand with random actions
    while state.status:
        # Track board state changes and street
        board_len = len(state.board_cards)
        
        # Detect when board changes and print street/cards
        if board_len > last_board_len:
            if board_len == 3:
                board_str = " ".join([f"{str(card[0].rank)}{str(card[0].suit)}" for card in state.board_cards])
                print(f"\n*** FLOP *** [{board_str}]")
            elif board_len == 4:
                turn_card = f"{str(state.board_cards[3][0].rank)}{str(state.board_cards[3][0].suit)}"
                board_str = " ".join([f"{str(card[0].rank)}{str(card[0].suit)}" for card in state.board_cards])
                print(f"\n*** TURN *** [{turn_card}] (Board: {board_str})")
            elif board_len == 5:
                river_card = f"{str(state.board_cards[4][0].rank)}{str(state.board_cards[4][0].suit)}"
                board_str = " ".join([f"{str(card[0].rank)}{str(card[0].suit)}" for card in state.board_cards])
                print(f"\n*** RIVER *** [{river_card}] (Board: {board_str})")
            last_board_len = board_len
        
        # Determine current street for action display
        street_name = "PRE-FLOP"
        if board_len == 3:
            street_name = "FLOP"
        elif board_len == 4:
            street_name = "TURN"
        elif board_len == 5:
            street_name = "RIVER"

        # Check what actions are available
        if state.actor_indices:  # If there's an active player who needs to act
            available_actions = []

            # Check if folding is possible
            if state.can_fold():
                available_actions.append("fold")

            # Check if checking/calling is possible
            if state.can_check_or_call():
                available_actions.append("check_or_call")

            # Check if betting/raising is possible
            if state.can_complete_bet_or_raise_to():
                available_actions.append("bet_or_raise")

            # Choose a random action
            if available_actions:
                action = np.random.choice(available_actions)
                actor = state.actor_indices[0]

                # Print the action
                action_str = ""
                if action == "fold":
                    state.fold()
                    action_str = "folds"
                elif action == "check_or_call":
                    call_amount = state.checking_or_calling_amount
                    state.check_or_call()
                    action_str = (
                        "checks" if call_amount == 0 else f"calls {call_amount}"
                    )
                elif action == "bet_or_raise":
                    min_amount = state.min_completion_betting_or_raising_to_amount
                    state.complete_bet_or_raise_to()
                    action_str = f"bets/raises to {min_amount}"

                print(
                    f"[{street_name}] Player {actor}: {action_str} [Pot: {state.total_pot_amount}]"
                )
                action_count += 1
        else:
            # No player actions needed, state will auto-advance
            break

    print(f"\n--- FINAL BOARD ---")
    if len(state.board_cards) > 0:
        final_board = " ".join(
            [f"{str(card[0].rank)}{str(card[0].suit)}" for card in state.board_cards]
        )
    else:
        final_board = "EMPTY"
    print(f"Board: {final_board}")

    print(f"\n--- RESULTS ---")
    print(f"Final stacks: {state.stacks}")
    print(f"Payoffs: {state.payoffs}")

    # Show winning hands
    print(f"\nWinner at showdown:")
    for i, is_active in enumerate(state.statuses):
        if is_active:
            print(f"  Player {i}: {state.hole_cards[i]}")
    
    # Rotate stacks for next hand (button moves clockwise)
    # Move position 0 to the end, everyone else shifts left
    current_stacks = list(state.stacks[1:]) + [state.stacks[0]]

Simulating hands: 100%|██████████| 10/10 [00:00<00:00, 601.98it/s]


HAND 1 - Button at position 0
Starting stacks: [998, 996, 1000, 1000, 1000]

--- HOLE CARDS ---
Player 0: [5h, 9h]
Player 1: [7s, 7h]
Player 2: [2c, Ah]
Player 3: [2h, As]
Player 4: [Kd, 7c]

--- GAME PLAY ---
[PRE-FLOP] Player 2: calls 4 [Pot: 10]
[PRE-FLOP] Player 3: bets/raises to 8 [Pot: 18]
[PRE-FLOP] Player 4: bets/raises to 12 [Pot: 30]
[PRE-FLOP] Player 0: bets/raises to 16 [Pot: 44]
[PRE-FLOP] Player 1: folds [Pot: 44]
[PRE-FLOP] Player 2: folds [Pot: 44]
[PRE-FLOP] Player 3: folds [Pot: 44]
[PRE-FLOP] Player 4: folds [Pot: 0]

--- FINAL BOARD ---
Board: EMPTY

--- RESULTS ---
Final stacks: [1028, 996, 996, 992, 988]
Payoffs: [28, -4, -4, -8, -12]

Winner at showdown:
  Player 0: [5h, 9h]

HAND 2 - Button at position 1
Starting stacks: [994, 992, 992, 988, 1028]

--- HOLE CARDS ---
Player 0: [8h, 6s]
Player 1: [4c, Kc]
Player 2: [8d, 4d]
Player 3: [Th, 8c]
Player 4: [7d, Qh]

--- GAME PLAY ---
[PRE-FLOP] Player 2: bets/raises to 8 [Pot: 14]
[PRE-FLOP] Player 3: calls 8 [Pot: 

In [62]:
# Calculate hand strength of AsKs on QsJs4h flop
# This calculates the probability of winning against a random hand
from pokerkit import calculate_hand_strength

with ProcessPoolExecutor() as executor:
    strength = calculate_hand_strength(
        2,  # number of active players (us vs 1 opponent)
        parse_range("AsKs"),  # our hand
        Card.parse("QsJs4h"),  # board (flop)
        2,  # hole cards per player
        5,  # total board cards (we have 3, need 2 more for turn/river)
        Deck.STANDARD,  # standard 52-card deck
        (StandardHighHand,),  # hand type
        sample_count=50,  # number of Monte Carlo simulations
        executor=executor,
    )

print(f"AsKs hand strength on QsJs4h flop: {strength:.1%}")
print(f"(Probability of beating a random hand)")

AsKs hand strength on QsJs4h flop: 70.0%
(Probability of beating a random hand)
